# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,16.01,80,7,1.79,NZ,1738266874
1,1,vezirkopru,41.1436,35.4547,9.75,46,100,0.18,TR,1738266875
2,2,belyy yar,53.6039,91.3903,-22.06,92,100,1.00,RU,1738266877
3,3,grand-bassam,5.2118,-3.7388,28.98,80,3,3.05,CI,1738266878
4,4,papatowai,-46.5619,169.4708,14.99,63,96,0.50,NZ,1738266879


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# citties = city_data_df["City"]
# lng = city_data_df["Lng"]
# lat = city_data_df["Lat"]
# map_plot = city_data_df.hvplot.scatter(
#     'Longitude', 'Latitude',
#     size='Humidity',
#     color='Humidity',
#     geo=True,
#     tiles='OSM',
#     hover_cols=['City', 'Humidity']
# )

# map_plot
city_data_df.hvplot.scatter(
    x='Lng', y='Lat', color='City', size=10, alpha=0.5, tiles='OSM', 
    title='City Data Map',
    xlabel='Longitude',
    ylabel='Latitude',
    width= 900,
    height= 400,
)

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [x]   (y,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition =(city_data_df['Max Temp'].between(21, 27))
wind_condition =(city_data_df['Wind Speed']<4.5)
cloud_condition =(city_data_df['Cloudiness']==0)

filterd_cities = city_data_df[temp_condition & wind_condition & cloud_condition]

# Drop any rows with null values
cleaned_cities = filterd_cities.dropna()

# Display sample data
cleaned_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,inhambane,-23.8650,35.3833,24.55,81,0,3.16,MZ,1738266897
56,56,chonchi,-42.6128,-73.8092,21.76,47,0,4.29,CL,1738266945
218,218,callaway,30.1530,-85.5699,22.80,72,0,3.09,US,1738267145
247,247,topes de collantes,21.9139,-80.0189,25.08,36,0,4.46,CU,1738267183
281,281,aswan,24.0934,32.9070,21.61,28,0,3.60,EG,1738267223
298,298,taoudenni,22.6783,-3.9836,21.79,13,0,3.71,ML,1738267243
321,321,cabo san lucas,22.8909,-109.9124,23.75,24,0,1.79,MX,1738267028


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,80,
1,vezirkopru,TR,41.1436,35.4547,46,
2,belyy yar,RU,53.6039,91.3903,92,
3,grand-bassam,CI,5.2118,-3.7388,80,
4,papatowai,NZ,-46.5619,169.4708,63,
...,...,...,...,...,...,...
578,reggane,DZ,26.7158,0.1714,17,
579,vilassar de mar,ES,41.5096,2.3937,65,
580,kucove,AL,40.8003,19.9167,81,
581,berezovyy,RU,51.6667,135.7000,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,#Only need the first result
    "apiKey":geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    # response.raise_for_status()
    if response.status_code != 200:
        print(f"Error: {response.stauts_code}, {response.text}")
        hotel.df.loc[index, "Hotel Name"] = "API request failed"
        continue 
    
    # Convert the API response to JSON format
    name_address = response.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waitangi - nearest hotel: Hotel Chathams
vezirkopru - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
grand-bassam - nearest hotel: Hôtel 3G
papatowai - nearest hotel: No hotel found
villa gesell - nearest hotel: Hotel El Faro
arraial do cabo - nearest hotel: No hotel found
lincoln - nearest hotel: New Victorian Suites Lincoln
blackmans bay - nearest hotel: Villa Howden
pingzhuang - nearest hotel: No hotel found
rawson - nearest hotel: Hotel Deportivo
drayton valley - nearest hotel: Lakeview Inn & Suites
grytviken - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
olonkinbyen - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
ust-nera - nearest hotel: гостиница "Солнечная"
vorgashor - nearest hotel: София
constantia - nearest hotel: Hotel Dali
inhambane - nearest hotel: Hotel de Inhambane
khandyga - nearest hotel: Апельсин
port alfred - nearest hotel: No hotel found
sishui - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,80,Hotel Chathams
1,vezirkopru,TR,41.1436,35.4547,46,No hotel found
2,belyy yar,RU,53.6039,91.3903,92,No hotel found
3,grand-bassam,CI,5.2118,-3.7388,80,Hôtel 3G
4,papatowai,NZ,-46.5619,169.4708,63,No hotel found
...,...,...,...,...,...,...
578,reggane,DZ,26.7158,0.1714,17,No hotel found
579,vilassar de mar,ES,41.5096,2.3937,65,No hotel found
580,kucove,AL,40.8003,19.9167,81,Hotel Familja
581,berezovyy,RU,51.6667,135.7000,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.scatter(
    x='Lng', y='Lat', color='City', size=10, alpha=0.5, tiles='OSM', 
    title='City Data Map',
    xlabel='Longitude',
    ylabel='Latitude',
    width= 900,
    height= 400,
    hover_cols=['Country', 'Hotel Name', 'Lat', 'Lng']
)

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [x]   (y,City,Country,Hotel Name,Lat,Lng)